In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import re
import string
from nltk.corpus import stopwords
import unicodedata
from fuzzywuzzy import process, fuzz

tqdm.pandas()

## Concatenation and Saving Final Corpus

## Corpus Preprocessing using NLP Techniques

In [2]:
# Define the core path
PATH = 'E:/Software/Data Science and AI/NLP/Edliyye/Legal Acts Question Answering/NLP project'

text_path = os.path.join(PATH, 'full_qanun_text_chunks.parquet')

data = pd.read_parquet(text_path)

display(data.head())
display(data.tail())
data.info()
display(len(data['text'].value_counts()))

,text,unique_word_count
0,azərbaycan respublikası mülki məcəlləsinin təs...,13
1,azərbaycan respublikasının mülki məcəlləsi təs...,6
2,azərbaycan respublikasının mülki məcəlləsi qüv...,24
3,azərbaycan respublikasının mülki məcəlləsinə d...,35
4,azərbaycan ssr mülki mülki prosessual məcəlləl...,17


,text,unique_word_count
1500188,maddədə istehsal məişət tullantıları sözləri t...,8
1500189,maddədə istehsal məişət tullantıları sözləri t...,9
1500190,maddədə sənaye məişət tullantılarının sözləri ...,9
1500191,qiymətli metallar qiymətli daşlar azərbaycan r...,19
1500192,yaşıllıqların mühafizəsi azərbaycan respublika...,18


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500193 entries, 0 to 1500192
Data columns (total 2 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   text               1500193 non-null  object
 1   unique_word_count  1500193 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 22.9+ MB


967905

## Preprocessing Text Column

In [3]:
data.loc[data['unique_word_count'] > 512]

,text,unique_word_count


In [6]:
display(len(data['text'].value_counts()))
display(data['text'].head())

967905

0    azərbaycan respublikası mülki məcəlləsinin təs...
1    azərbaycan respublikasının mülki məcəlləsi təs...
2    azərbaycan respublikasının mülki məcəlləsi qüv...
3    azərbaycan respublikasının mülki məcəlləsinə d...
4    azərbaycan ssr mülki mülki prosessual məcəlləl...
Name: text, dtype: object

In [ ]:
# Remove Leading/Trailing Spaces
def g(text):
    return text.strip()

data['text'] = data['text'].apply(g)

In [ ]:
# Replace Multiple Spaces with a Single One
def g(text):
    return re.sub(' +', ' ', text)

data['text'] = data['text'].apply(g)

In [ ]:
# Function to normalize text
def normalize_text(text):
    return unicodedata.normalize('NFC', text)

# Apply normalization to 'text' column
data['text'] = data['text'].apply(normalize_text)

display(len(data['text'].value_counts()))
display(data['text'].head())

In [ ]:
# Remove "\xa0" pattern
data['text'] = data['text'].str.replace(r'\xa0', '', regex=True)

In [ ]:
# Replace newline characters
data['text'] = data['text'].str.replace('\n', ' ', regex=True)

In [ ]:
data['text'].loc[data['text'].str.contains('_____________')]

In [4]:
data['text'].iloc[500]

'5. 30 sentyabr 2014-cü il tarixli 1047-ivqd nömrəli azərbaycan respublikasının qanunu (“azərbaycan” qəzeti, 9 noyabr 2014-cü il, № 245, azərbaycan respublikasının qanunvericilik toplusu, 2014-cü il, № 10, maddə 1167)'

***Given the importance of preserving the original text, including punctuation, for the semantic search model on Azerbaijani legislation, we should not remove punctuation. This is crucial for accurately matching legal texts, as punctuation can change the meaning of the sentences and affect the search results, because users could write or copy/paste the legal acts as they are in original, including numbers of articles and other symbols!***

In [ ]:
# Remove lines of underscores
data['text'] = data['text'].str.replace(r'_{2,}', '', regex=True)

In [5]:
def del_symbols(data:str):
    data = re.sub(r'[^a-zA-ZəƏıIöÖüÜğĞçÇşŞ]', ' ', data)
    data = re.sub(r'\s+', ' ', data)
    data = data.strip()
    return data

data["text"] = data["text"].progress_apply(del_symbols)

display(len(data['text'].value_counts()))
display(data['text'].head())

100%|█████████████████████████████████████████████████████████████████████| 1501973/1501973 [00:40<00:00, 37022.11it/s]


990991

0    azərbaycan respublikası mülki məcəlləsinin təs...
1    azərbaycan respublikasının mülki məcəlləsi təs...
2    azərbaycan respublikasının mülki məcəlləsi qüv...
3    azərbaycan respublikasının mülki məcəlləsinə d...
4    azərbaycan ssr mülki və mülki prosessual məcəl...
Name: text, dtype: object

In [ ]:
data['text'] = data['text'].str.split().apply(lambda x: [word for word in x if len(word) > 2]).str.join(' ')

In [9]:
data['text'].iloc[500]

'sentyabr tarixli azərbaycan respublikasının qanunu azərbaycan qəzeti noyabr azərbaycan respublikasının qanunvericilik toplusu'

In [15]:
# Load Azerbaijani stopwords and add additional ones
az_stopwords = list(stopwords.words("azerbaijani"))
az_stopwords.append("i")
az_stopwords.append("b")
az_stopwords.append("ivqd")
az_stopwords.append("maddə")
az_stopwords.append("nömrəli")
az_stopwords.append("si")

# Function to remove stopwords
def del_stopwords(data: str):
    data = " ".join([i for i in data.split() if i not in az_stopwords])
    return data

data["text"] = data["text"].progress_apply(del_stopwords)

display(len(data['text'].value_counts()))
display(data['text'].head())

100%|█████████████████████████████████████████████████████████████████████| 1501973/1501973 [01:12<00:00, 20760.47it/s]


970801

0    azərbaycan respublikası mülki məcəlləsinin təs...
1    azərbaycan respublikasının mülki məcəlləsi təs...
2    azərbaycan respublikasının mülki məcəlləsi qüv...
3    azərbaycan respublikasının mülki məcəlləsinə d...
4    azərbaycan ssr mülki mülki prosessual məcəlləl...
Name: text, dtype: object

In [ ]:
# Function to count unique words in a text
def count_unique_words(text):
    words = text.split()
    unique_words = set(words)
    return len(unique_words)

# Function to split text into chunks with a maximum of max_unique_words unique words
def split_text_into_chunks(text, max_unique_words=512):
    words = text.split()
    chunks = []
    chunk = []
    unique_words = set()

    for word in words:
        if word not in unique_words:
            if len(unique_words) >= max_unique_words:
                chunks.append(' '.join(chunk))
                chunk = []
                unique_words = set()
        chunk.append(word)
        unique_words.add(word)

    if chunk:
        chunks.append(' '.join(chunk))
    
    return chunks

# Apply the function to split texts into chunks and flatten the list of chunks into a new DataFrame
rows = []
for _, row in data.iterrows():
    chunks = split_text_into_chunks(row['text'])
    for chunk in chunks:
        rows.append({'text': chunk, 'unique_word_count': count_unique_words(chunk)})

chunked_data = pd.DataFrame(rows)

# Ensure each chunk has at most 512 unique words
chunked_data = chunked_data[chunked_data['unique_word_count'] <= 512]

# Output the chunked_data DataFrame
display(chunked_data)

#chunked_data.to_parquet('./full_qanun_text_chunks.parquet', index=False)

In [11]:
data.to_parquet('full_qanun_text.parquet', index=False)

## Old Preprocessing Version

In [8]:
def del_symbols(data:str):
    data = re.sub(r'[^a-zA-ZəƏıIöÖüÜğĞçÇşŞ]', ' ', data)
    data = re.sub(r'\s+', ' ', data)
    return data

In [9]:
data

,text,len,document_name,document_type
0,Azərbaycan Respublikası Mülki\nMəcəlləsinin tə...,130,Azərbaycan Respublikası Mülki\nMəcəlləsinin tə...,Azərbaycan RespublİkasINIn Qanunu [1]
1,Azərbaycan RespublİkasINIn Qanunu [1],37,Azərbaycan Respublikası Mülki\nMəcəlləsinin tə...,Azərbaycan RespublİkasINIn Qanunu [1]
2,Maddə\n1,7,Azərbaycan Respublikası Mülki\nMəcəlləsinin tə...,Azərbaycan RespublİkasINIn Qanunu [1]
3,Azərbaycan\nRespublikasının Mülki Məcəlləsi tə...,58,Azərbaycan Respublikası Mülki\nMəcəlləsinin tə...,Azərbaycan RespublİkasINIn Qanunu [1]
4,Maddə\n2,7,Azərbaycan Respublikası Mülki\nMəcəlləsinin tə...,Azərbaycan RespublİkasINIn Qanunu [1]
...,...,...,...,...
1404621,İsfəndiyar Bəxtiyar oğlu Vahabzadə Azərbaycan\...,133,İ.B.Vahabzadənin\nAzərbaycan Respublikasının M...,AZƏRBAYCAN\nRESPUBLİKASI PREZİDENTİNİN SƏRƏNCAMI
1404622,"İlham ƏLİYEV,",13,İ.B.Vahabzadənin\nAzərbaycan Respublikasının M...,AZƏRBAYCAN\nRESPUBLİKASI PREZİDENTİNİN SƏRƏNCAMI
1404623,Azərbaycan\nRespublikasının Prezidenti,37,İ.B.Vahabzadənin\nAzərbaycan Respublikasının M...,AZƏRBAYCAN\nRESPUBLİKASI PREZİDENTİNİN SƏRƏNCAMI
1404624,"Bakı şəhəri, 27 iyun 2005-ci il",31,İ.B.Vahabzadənin\nAzərbaycan Respublikasının M...,AZƏRBAYCAN\nRESPUBLİKASI PREZİDENTİNİN SƏRƏNCAMI


In [10]:
data["text"] = data["text"].progress_apply(del_symbols)
data["document_name"] = data["document_name"].progress_apply(del_symbols)
data["document_type"] = data["document_type"].progress_apply(del_symbols)
data["len"] = data["text"].apply(lambda x: len(x.split()))

100%|█████████████████████████████████████████████████████████████████████| 1404626/1404626 [00:16<00:00, 83643.38it/s]


In [11]:
data = data.loc[data["len"] > 5].reset_index(drop=True)

In [12]:
data = data.loc[data["text"] != data["document_name"]].reset_index(drop=True)
data = data.loc[data["text"] != data["document_type"]].reset_index(drop=True)

In [16]:
az_stopwords = list(stopwords.words("azerbaijani"))
az_stopwords.append("i")
az_stopwords.append("b")

In [17]:
def del_stopwords(data:str):
    data = " ".join([i for i in data.split() if i not in az_stopwords])
    return data

In [18]:
data["text"] = data["text"].progress_apply(del_stopwords)
data["len"] = data["text"].apply(lambda x: len(x.split()))

100%|███████████████████████████████████████████████████████████████████████| 949282/949282 [01:11<00:00, 13290.84it/s]


In [19]:
data

,text,len,document_name,document_type
0,azərbaycan respublikasının mülki məcəlləsi təs...,6,azərbaycan respublikası mülki məcəlləsinin təs...,azərbaycan respubli kasinin qanunu
1,azərbaycan respublikasının mülki məcəlləsi qüv...,36,azərbaycan respublikası mülki məcəlləsinin təs...,azərbaycan respubli kasinin qanunu
2,azərbaycan respublikasının mülki məcəlləsinə d...,44,azərbaycan respublikası mülki məcəlləsinin təs...,azərbaycan respubli kasinin qanunu
3,azərbaycan ssr mülki mülki prosessual məcəlləl...,24,azərbaycan respublikası mülki məcəlləsinin təs...,azərbaycan respubli kasinin qanunu
4,azərbaycan ssr mülki mülki prosessual məcəlləl...,33,azərbaycan respublikası mülki məcəlləsinin təs...,azərbaycan respubli kasinin qanunu
...,...,...,...,...
949277,tofiq nadir oğlu zülfüqarov azərbaycan respubl...,13,t n zülfüqarovun azərbaycan respublikasının la...,azərbaycan respubli kasi prezi denti ni n sərə...
949278,azərbaycan respublikası konstitusiyasının madd...,9,azərbaycan respublikasının misir ərəb respubli...,azərbaycan respubli kasi prezi denti ni n sərə...
949279,azərbaycan respublikasının misir ərəb respubli...,31,azərbaycan respublikasının misir ərəb respubli...,azərbaycan respubli kasi prezi denti ni n sərə...
949280,azərbaycan respublikası konstitusiyasının madd...,9,i b vahabzadənin azərbaycan respublikasının mo...,azərbaycan respubli kasi prezi denti ni n sərə...


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949282 entries, 0 to 949281
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   text           949282 non-null  object
 1   len            949282 non-null  int64 
 2   document_name  949282 non-null  object
 3   document_type  949282 non-null  object
dtypes: int64(1), object(3)
memory usage: 29.0+ MB
